In [48]:
pip install accelerate -U evaluate scikit-learn


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 71.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 42.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [45]:
from transformers import AutoTokenizer
from datasets import load_dataset
from huggingface_hub import notebook_login
import evaluate

In [3]:
notebook_login()

In [91]:
checkpoint = "distilbert/distilbert-base-uncased"

In [92]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
dataset = load_dataset("mwarchalowski/names-data")

Generating train split:   0%|          | 0/4683999 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/520445 [00:00<?, ? examples/s]

In [7]:
dataset = load_dataset("mwarchalowski/names-data", split="train[:10%]")

In [8]:
dataset = dataset.filter(lambda e: e["name"] is not None)

Filter:   0%|          | 0/468400 [00:00<?, ? examples/s]

In [93]:
def tokenize_function(examples):
    return tokenizer(examples["name"], truncation=True)
    

In [94]:
dataset

Dataset({
    features: ['name', 'label'],
    num_rows: 468400
})

In [95]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/468400 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [107]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [97]:
samples = tokenized_datasets[:8]
samples.items()

dict_items([('name', ['Murphy Facility Solutions', 'Mansell Building Solutions', 'Johnny Boulos', 'Gurdeep Singh', 'Stefan Birrer', 'ECG Egineering Consulting Group', 'Topdeck Travel', 'Tamar Stiebel']), ('label', [0, 0, 1, 1, 1, 0, 0, 1]), ('input_ids', [[101, 7104, 4322, 7300, 102], [101, 16042, 5349, 2311, 7300, 102], [101, 5206, 8945, 18845, 2015, 102], [101, 19739, 25547, 13699, 5960, 102], [101, 8852, 12170, 14343, 2099, 102], [101, 14925, 2290, 1041, 11528, 20550, 10552, 2177, 102], [101, 2327, 26547, 3604, 102], [101, 17214, 2906, 2358, 2666, 8671, 102]]), ('attention_mask', [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]])])

In [99]:
samples = {k: v for k, v in samples.items() if k not in ["name", "label"]}

In [100]:
batch = data_collator(samples)
batch

{'input_ids': tensor([[  101,  7104,  4322,  7300,   102,     0,     0,     0,     0],
        [  101, 16042,  5349,  2311,  7300,   102,     0,     0,     0],
        [  101,  5206,  8945, 18845,  2015,   102,     0,     0,     0],
        [  101, 19739, 25547, 13699,  5960,   102,     0,     0,     0],
        [  101,  8852, 12170, 14343,  2099,   102,     0,     0,     0],
        [  101, 14925,  2290,  1041, 11528, 20550, 10552,  2177,   102],
        [  101,  2327, 26547,  3604,   102,     0,     0,     0,     0],
        [  101, 17214,  2906,  2358,  2666,  8671,   102,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0]])}

In [101]:
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 9]), 'attention_mask': torch.Size([8, 9])}

In [102]:
from transformers import TrainingArguments

training_args = TrainingArguments("distillbert-names-data", fp16=True, per_device_train_batch_size=600, evaluation_strategy="epoch")


In [103]:
from transformers import AutoModelForSequenceClassification

In [104]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [108]:
train_validation_set = tokenized_datasets.train_test_split(test_size=0.1)
train_validation_set = train_validation_set.remove_columns(["name"])
train_validation_set.rename_column("label", "labels")

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 421560
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 46840
    })
})

In [109]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_validation_set["train"],
    eval_dataset=train_validation_set["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)


In [110]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.073900,0.058157
2,0.048600,0.055600
3,0.032200,0.058935


TrainOutput(global_step=2109, training_loss=0.04793074980003679, metrics={'train_runtime': 253.4494, 'train_samples_per_second': 4989.871, 'train_steps_per_second': 8.321, 'total_flos': 5713805307762720.0, 'train_loss': 0.04793074980003679, 'epoch': 3.0})

In [23]:
predictions = trainer.predict(train_validation_set["test"])

In [24]:
print(predictions.predictions.shape, predictions.label_ids.shape)

(46840, 2) (46840,)


In [62]:
predictions

PredictionOutput(predictions=array([[ 3.3574219, -3.3613281],
       [-2.3554688,  2.8105469],
       [-2.7109375,  3.1113281],
       ...,
       [-3.4589844,  3.7246094],
       [ 4.84375  , -4.9335938],
       [-2.8984375,  3.2949219]], dtype=float32), label_ids=array([0, 1, 1, ..., 1, 0, 1]), metrics={'test_loss': 0.054905541241168976, 'test_runtime': 54.8348, 'test_samples_per_second': 854.203, 'test_steps_per_second': 106.775})

In [26]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [27]:
preds

array([0, 1, 1, ..., 1, 0, 1])

In [28]:
predictions.label_ids

array([0, 1, 1, ..., 1, 0, 1])

In [39]:
z = zip(preds, predictions.label_ids)

In [38]:
negs = [x for x,y in z if x != y]

In [40]:
pos = [x for x,y in z if x == y]

In [41]:
len(negs)

682

In [42]:
len(pos)

46158

In [43]:
1 - 675 / 46165

0.9853785335210657

In [49]:
accuracy = evaluate.load("accuracy")

In [50]:
accuracy.description

'\nAccuracy is the proportion of correct predictions among the total number of cases processed. It can be computed with:\nAccuracy = (TP + TN) / (TP + TN + FP + FN)\n Where:\nTP: True positive\nTN: True negative\nFP: False positive\nFN: False negative\n'

In [51]:
accuracy.features

{'predictions': Value(dtype='int32', id=None),
 'references': Value(dtype='int32', id=None)}

In [52]:
accuracy.compute(references=predictions.label_ids, predictions=preds)

{'accuracy': 0.9854397950469684}

In [53]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

In [54]:
clf_metrics.compute(references=predictions.label_ids, predictions=preds)

{'accuracy': 0.9854397950469684,
 'f1': 0.9796052631578948,
 'precision': 0.9741287022719163,
 'recall': 0.9851437507518345}

In [58]:
[print(model.description) for model in clf_metrics.evaluation_modules]


Accuracy is the proportion of correct predictions among the total number of cases processed. It can be computed with:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
 Where:
TP: True positive
TN: True negative
FP: False positive
FN: False negative


The F1 score is the harmonic mean of the precision and recall. It can be computed with the equation:
F1 = 2 * (precision * recall) / (precision + recall)


Precision is the fraction of correctly labeled positive examples out of all of the examples that were labeled as positive. It is computed via the equation:
Precision = TP / (TP + FP)
where TP is the True positives (i.e. the examples correctly labeled as positive) and FP is the False positive examples (i.e. the examples incorrectly labeled as positive).


Recall is the fraction of the positive examples that were correctly labeled by the model as positive. It can be computed with the equation:
Recall = TP / (TP + FN)
Where TP is the true positives and FN is the false negatives.



[None, None, None, None]

In [111]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [112]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_validation_set["train"],
    eval_dataset=train_validation_set["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [114]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.028300,0.061525,0.983049,0.976321,0.971339,0.981355
2,0.023800,0.072336,0.983390,0.976855,0.969529,0.984293
3,0.013200,0.085540,0.983262,0.976654,0.970240,0.983153


TrainOutput(global_step=2109, training_loss=0.02053045395597455, metrics={'train_runtime': 272.9372, 'train_samples_per_second': 4633.593, 'train_steps_per_second': 7.727, 'total_flos': 6088698394974720.0, 'train_loss': 0.02053045395597455, 'epoch': 3.0})

In [72]:
trainer.push_to_hub("mwarchalowski/name-recognition")
# trainer.hub_model_id = "mwarchalowski/name-classifier"

CommitInfo(commit_url='https://huggingface.co/mwarchalowski/name-classifier/commit/dc245d8a39631dd1119ec21942935fe85fc4ba09', commit_message='mwarchalowski/name-recognition', commit_description='', oid='dc245d8a39631dd1119ec21942935fe85fc4ba09', pr_url=None, pr_revision=None, pr_num=None)

In [73]:
trainer.evaluate("Hi my name is")

AttributeError: 'list' object has no attribute 'keys'

In [77]:
trainer.hub_model_id

'mwarchalowski/name-classifier'

In [115]:
model = trainer.model

In [116]:
model.config.id2label

{0: 'LABEL_0', 1: 'LABEL_1'}

In [85]:
label2id = {'Organization': 0, 'Person': 1}

In [87]:
id2label = {0: 'Organization', 1: 'Person'}

In [117]:
model.config.label2id = label2id

In [118]:
model.config.id2label = id2label

In [120]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mwarchalowski/distillbert-names-data/commit/2f2bd7a77d0c381c76caa0d2abf5dbb48f647c0b', commit_message='End of training', commit_description='', oid='2f2bd7a77d0c381c76caa0d2abf5dbb48f647c0b', pr_url=None, pr_revision=None, pr_num=None)